# Goal: Who should we lend money to?

As we want to find insights to improve and guide decisions and underwriting criteria, we want features to help make better prediction for whether a person is able to repay the loan and the probability of default. I believe when we are personally thinking about lending money to someone, there are traits that can boost our confidence for the borrowers repaying the money and there are red flags that would make us hesitate to lend money to some people. Here are some characters that I would look at before lending money to someone and I will use this as a guideline and to see if some of those can be quantified with the data sets that we have.

What do I look for from a borrower:

    - being responsible
    
    - being trustworthy (good identity/status)
    
    - others
    
What are some ways that we can quantify each of those with the data that we have?

    - responsibilities:      
        - missed call vs incoming call?
        -
        - any text from Branch reminding for late payment? ***
        
    - trustworthiness:
        - number of people in their contact list
        - number of calls (from a contact vs non-contact)
        - length of time of the user digital history
        - number of people they contact very often

    - others:
        - constantly calling late at night?
        - constantly texting late at night? ***
        - average duration for phone call?
        - Number of devices?

In [169]:
import glob
import time

import pandas as pd
import numpy as np

from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('../logs/users.csv')
df.head(3)

,user_id,disbursed_at,loan_status
0,1,2017-03-02T00:00:00.000Z,repaid
1,2,2017-03-01T00:00:00.000Z,defaulted
2,3,2017-03-02T00:00:00.000Z,repaid


In [3]:
user_ids = df.user_id.values

# looking at different call type

In [160]:
start_time = time.time()
calls_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    call_logs = []
    call_logs += [glob.glob(d+'/collated_call_log.txt') for d in devices]
    incoming_call = 0
    outgoing_call = 0
    missed_call = 0
    total_call = 0
    for call_log in call_logs:
        if len(call_log) > 0:  # filter the users that don't have a call log
            df_call = pd.read_json(call_log[0])
            if len(df_call) > 0:  #  if not empty call logs, accumlate count
                total_call += len(df_call)
                incoming_call += len(df_call[df_call['call_type'] == 1])
                missed_call += len(df_call[df_call['call_type'] == 3])
                outgoing_call += len(df_call[df_call['call_type'] == 2])
    calls_data.append({'user_id':uid, 'total_incoming_calls':incoming_call*100.0/total_call, 
                       'total_missed_calls':missed_call*100.0/total_call, 
                       'total_outgoing_calls':outgoing_call*100.0/total_call})
print len(calls_data)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 14.41


In [162]:
calls_data = pd.DataFrame(calls_data)
calls_data = calls_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_calls = calls_data[['loan_status', 'total_incoming_calls', 'total_outgoing_calls','total_missed_calls']].groupby('loan_status').mean().reset_index()
grouped_calls.columns = ['loan_status', '%_incoming', '%_outgoing', '%_missed']
grouped_calls

,loan_status,%_incoming,%_outgoing,%_missed
0,defaulted,29.756039,56.383679,13.614776
1,repaid,30.160192,56.196026,13.416973


# Total number of calls:

In [174]:
start_time = time.time()
calls_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    call_logs = []
    call_logs += [glob.glob(d+'/collated_call_log.txt') for d in devices]
    total_call = 0
    for call_log in call_logs:
        if len(call_log) > 0:  # filter the users that don't have a call log
            df_call = pd.read_json(call_log[0])
            if len(df_call) > 0:  #  if not empty call logs, accumlate count
                total_call += len(df_call)
    calls_data.append({'user_id':uid, 'total_calls':total_call})
print len(calls_data)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 14.50


In [175]:
calls_data = pd.DataFrame(calls_data)
calls_data = calls_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_calls = calls_data[['loan_status', 'total_calls']].groupby('loan_status').mean().reset_index()
grouped_calls.columns = ['loan_status', 'total_calls']
grouped_calls

,loan_status,total_calls
0,defaulted,1053.935
1,repaid,2953.380


# Total number of text

In [177]:
start_time = time.time()
text_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    sms_logs = []
    sms_logs += [glob.glob(d+'/collated_sms_log.txt') for d in devices]
    total_text = 0
    for sms_log in sms_logs:
        if len(sms_log) > 0:
            df_sms = pd.read_json(sms_log[0])
            if len(df_sms) > 0:
                total_text += len(df_sms)
    text_data.append({'user_id':uid, 'total_text':total_text})
print len(text_data)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 19.49


In [180]:
text_data = pd.DataFrame(text_data)
text_data = text_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_text = text_data[['loan_status', 'total_text']].groupby('loan_status').mean().reset_index()
grouped_text.columns = ['loan_status', 'total_text']
grouped_text

,loan_status,total_text
0,defaulted,2233.935
1,repaid,5256.730


# Number of contacts in their contact list

Since a user can have multiple devices and have contact list on each of their devices, I am going to count the number of contacts they have without counting the duplicates.

In [36]:
start_time = time.time()
contacts_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    contact_lists = []
    contact_lists += [glob.glob(d+'/collated_contact_list.txt') for d in devices]
    contact_list_length = 0
    all_contacts = []
    for contact_list in contact_lists:
        if len(contact_list) > 0:  # filter the users that don't have a call log
            df_contact = pd.read_json(contact_list[0])
            if len(df_contact) > 0:  #  if not empty call logs, accumlate count
                all_contacts += df_contact['display_name'].values.tolist()
    unique_contact = len(set(all_contacts))
    contacts_data.append({'user_id':uid, 'unique_contact_count':unique_contact})
print 'runtime %.2f' %(time.time() - start_time)

runtime 5.80


In [51]:
contacts_data = pd.DataFrame(contacts_data)
contacts_data = contacts_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_contacts = contacts_data[['loan_status', 'unique_contact_count']].groupby('loan_status').agg(['min', 'mean','max', 'std']).reset_index()
grouped_contacts.columns = ['loan_status','min_unique_contact_count', 'avg_unique_contact_count', 'max_unique_contact_count', 'std']
grouped_contacts

,loan_status,min_unique_contact_count,avg_unique_contact_count,max_unique_contact_count,std
0,defaulted,7,294.465,1606,228.258625
1,repaid,0,417.385,2580,406.151450


# Number of calls from a contact vs non-contact

The idea here is to measure how connected the user is with the people aronud him, which I assume that if they contact each other often, the contact will be cached and appear in cached_name.

An unexpected problem with this measurement would be there are devices that don't have the field cached_name.

In [154]:
start_time = time.time()
calls_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    call_logs = []
    call_logs += [glob.glob(d+'/collated_call_log.txt') for d in devices]
    from_cached = 0
    not_from_cached = 0
    for call_log in call_logs:
        if len(call_log) > 0:  # filter the users that don't have a call log
            df_call = pd.read_json(call_log[0])
            if len(df_call) > 0:  #  if not empty call logs, accumlate count
                try:
                    from_cached += sum(df_call.cached_name.notnull())
                    not_from_cached += sum(df_call.cached_name.isnull())
                except:  # device that doesn't have cached_name as their field
                    continue
    calls_data.append({'user_id':uid, 'from_cached':from_cached, 'not_from_cached':not_from_cached})
print len(calls_data)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 17.78


In [155]:
calls_data = pd.DataFrame(calls_data)
calls_data = calls_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_calls = calls_data[['loan_status', 'from_cached', 'not_from_cached']].groupby('loan_status').mean().reset_index()
grouped_calls.columns = ['loan_status', 'from_cached', 'not_from_cached']
grouped_calls['%_from_cached'] = grouped_calls.from_cached*100.0/(grouped_calls.from_cached + grouped_calls.not_from_cached)
grouped_calls

,loan_status,from_cached,not_from_cached,%_from_cached
0,defaulted,672.155,244.54,73.323734
1,repaid,1908.585,611.99,75.720223


# length of time the user has digital history

This is similar to measuring the average account length in the credit score with a slight difference in people can have multiple credit cards and loans at the same time but when it comes to using phones or other digital devices, once they get a new phone, they don't go back to keep using their old one.

So here I adjust the calculation a little bit. Instead of taking the average of length of each device, I estimate it by taking the difference between the first day they call someone and the last day they call someone among all devices.

Note: Using call logs here becuase the datetime from sms logs can be very noisy (e.g. timestamp in 1970 or 2027).

In [96]:
import datetime
start_time = time.time()
digital_history = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    call_logs = []
    call_logs += [glob.glob(d+'/collated_call_log.txt') for d in devices]  
    dates = []  # storing the first day and last day in each log
    for call_log in call_logs:
        if len(call_log) > 0:  # filter the users that don't have a call log
            df_call = pd.read_json(call_log[0])
            if len(df_call) > 0:  #  if not empty call logs, accumlate count
                dates.append(min(df_call.datetime))
                dates.append(max(df_call.datetime))

    digital_history.append({'user_id':uid, 'first_day':min(dates), 'last_day':max(dates)})
print len(digital_history)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 18.69


In [105]:
digital_history = pd.DataFrame(digital_history)
digital_history = digital_history.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')
digital_history['length_of_history'] = digital_history.last_day - digital_history.first_day
digital_history['length_of_history'] = digital_history['length_of_history'].dt.days
digital_history
grouped_hist = digital_history[['loan_status', 'length_of_history']].groupby('loan_status').agg(['min', 'mean','max', 'std']).reset_index()
grouped_hist

loan_status length_of_history                           
                            min    mean    max         std
0   defaulted                 1  327.48   7352   774.43205
1      repaid                 0  579.80  13743  1193.79479

# number of people they contact very often

how many people has a use called more than 10 times?

After some looking into the data, I found that the average number of calls that user make to their contacts are around 10, so I set it as a threshold.

In [118]:
start_time = time.time()
contacts_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    contact_lists = []
    contact_lists += [glob.glob(d+'/collated_contact_list.txt') for d in devices]
    count_often_contact = 0
    for contact_list in contact_lists:
        if len(contact_list) > 0:  # filter the users that don't have a contact lists
            df_contact = pd.read_json(contact_list[0])
            if len(df_contact) > 0:  #  if not empty contact list, accumlate count
                count_often_contact += sum(df_contact['times_contacted'] > 10)
    contacts_data.append({'user_id':uid, 'count_often_contact':count_often_contact})
print 'runtime %.2f' %(time.time() - start_time)

runtime 5.68


In [119]:
contacts_data = pd.DataFrame(contacts_data)
contacts_data = contacts_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_contacts = contacts_data[['loan_status', 'count_often_contact']].groupby('loan_status').mean().reset_index()
grouped_contacts

,loan_status,count_often_contact
0,defaulted,36.980
1,repaid,54.405


# Number of calls late at night?

Let's define late at nigth mean 10pm - 4am. 

In [166]:
start_time = time.time()
calls_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    call_logs = []
    call_logs += [glob.glob(d+'/collated_call_log.txt') for d in devices]
    late_night_call = 0
    normal_hour_call = 0
    for call_log in call_logs:
        if len(call_log) > 0:  # filter the users that don't have a call log
            df_call = pd.read_json(call_log[0])
            if len(df_call) > 0:  #  if not empty call logs, accumlate count
                try:
                    df_call['hour'] = df_call['datetime'].dt.hour
                    late_night_call += sum(df_call['hour'] <= 4) + sum(df_call['hour'] >= 22)             
                    normal_hour_call += sum(df_call['hour'] < 22) - sum(df_call['hour'] <= 4)
                except:
                    continue
    calls_data.append({'user_id':uid, 'late_night_call':late_night_call, 'normal_hour_call':normal_hour_call})
print len(calls_data)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 23.21


In [167]:
calls_data = pd.DataFrame(calls_data)
calls_data = calls_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_calls = calls_data[['loan_status', 'late_night_call', 'normal_hour_call']].groupby('loan_status').mean().reset_index()
grouped_calls.columns = ['loan_status', 'late_night_call', 'normal_hour_call']
grouped_calls['%_of_late_night_call'] = grouped_calls.late_night_call*100.0/(grouped_calls.late_night_call + grouped_calls.normal_hour_call)
grouped_calls

,loan_status,late_night_call,normal_hour_call,%_of_late_night_call
0,defaulted,58.160,995.775,5.518367
1,repaid,169.285,2774.130,5.751313


It seems like the % of late night call among people who defaulted and repaid are pretty similar

# Average duration of phone call

In [139]:
start_time = time.time()
calls_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    call_logs = []
    call_logs += [glob.glob(d+'/collated_call_log.txt') for d in devices]
    durations = []
    for call_log in call_logs:
        if len(call_log) > 0:  # filter the users that don't have a call log
            df_call = pd.read_json(call_log[0])
            if len(df_call) > 0:  #  if not empty call logs, accumlate count
                durations += df_call.duration.values.tolist()
    calls_data.append({'user_id':uid, 'average_call_duration':np.mean(durations)})
print len(calls_data)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 15.69


In [140]:
calls_data = pd.DataFrame(calls_data)
calls_data = calls_data.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')

grouped_calls = calls_data[['loan_status', 'average_call_duration']].groupby('loan_status').agg(['min', 'mean','max', 'std']).reset_index()
grouped_calls

loan_status average_call_duration                                  
                                min       mean         max        std
0   defaulted              6.490637  40.180981  254.218045  26.780274
1      repaid              5.222051  43.014949  120.520807  19.844953

# Number of devices

In [148]:
start_time = time.time()
number_of_devices = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    number_of_devices.append({'user_id':uid, 'number_of_devices':len(devices)})
print len(number_of_devices)
print 'runtime %.2f' %(time.time() - start_time)

400
runtime 0.08


In [153]:
number_of_devices = pd.DataFrame(number_of_devices)
number_of_devices = number_of_devices.merge(df[['user_id', 'loan_status']], how = 'inner', on = 'user_id')
grouped_devices = number_of_devices[['loan_status', 'number_of_devices']].groupby('loan_status').agg(['min', 'mean','max', 'std']).reset_index()
grouped_devices

loan_status number_of_devices                     
                            min   mean max       std
0   defaulted                 1  1.195   4  0.477614
1      repaid                 1  1.500  10  1.177682

In [ ]:
# Any text from Branch about late payment?

# For this hypothesis, I will filter the sms logs by sms_address = 'Branch-Co' 
# and the message body contains the word 'late'

branch_late_data = []
for uid in user_ids:
    devices = glob.glob('../logs/user-%d/*' %uid)
    sms_logs = []
    sms_logs += [glob.glob(d+'/collated_sms_log.txt') for d in devices]
    branch_late_text = 0
    for sms_log in sms_logs:
        if len(sms_log) > 0:  # filter the users that don't have a sms log
            df_sms = pd.read_json(sms_log[0])
            if len(df_sms) > 0:  #  if not empty sms logs, accumlate count
                df_sms['sms_address'] = df_sms['sms_address'].to_string()
                from_branch = df_sms[df_sms['sms_address'] == 'Branch-Co']
                count_late_from_branch += sum(from_branch['message_body'].str.contains('late'))
        del df_sms, from_branch
    branch_late_data.append({'user_id':uid, 'total_late_text_from_branch':count_late_from_branch})
print len(branch_late_data)